10.	Perform the following operations in python on given dataset 
[employee_info.csv: Contains employee demographic and department data
performance.csv: Contains performance review scores]
a.	Clean invalid ages, join dates, and department names.
b.	Combine datasets using EmployeeID.
c.	Create performance average scores.
d.	Bucket performance into categories (Low/Medium/High).
e.	Correct mismatched or blank department entries.

In [1]:
import pandas as pd
import numpy as np

In [2]:
employee_df = pd.read_csv('employee_info.csv')
performance_df = pd.read_csv('performance.csv')

In [19]:
employee_df

,EmployeeID,Name,Department,Age,JoinDate
0,301,John,Hr,34.0,2015-06-01
1,302,Jane,Engineering,29.0,2018-09-15
2,303,Mike,Sales,40.0,2016-01-20
3,304,Sara,Sales,26.0,2020-11-30


In [21]:
performance_df

,EmployeeID,ReviewScore1,ReviewScore2,EligiblePromotion
0,301,3.5,3.8,1
1,302,4.2,4.5,1
2,303,2.8,2.5,0
3,304,4.5,4.7,1
4,306,3.0,3.1,0


# a. Clean invalid ages, join dates, and department names

In [23]:
employee_df['Age'] = pd.to_numeric(employee_df['Age'], errors='coerce')
employee_df.loc[(employee_df['Age'] < 18) | (employee_df['Age'] > 70), 'Age'] = np.nan
employee_df['Age'].fillna(employee_df['Age'].median(), inplace=True)

C:\Users\Sonali\AppData\Local\Temp\ipykernel_35052\3782032090.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  employee_df['Age'].fillna(employee_df['Age'].median(), inplace=True)


In [25]:
employee_df['JoinDate'] = pd.to_datetime(employee_df['JoinDate'], errors='coerce')
employee_df.dropna(subset=['JoinDate'], inplace=True)

In [6]:
employee_df['Department'] = employee_df['Department'].str.strip().str.title()

# b. Combine datasets using EmployeeID

In [27]:
merged_df = pd.merge(employee_df, performance_df, on="EmployeeID", how="inner")

# c. Create performance average scores

In [29]:
merged_df['AvgScore'] = merged_df[['ReviewScore1', 'ReviewScore2']].mean(axis=1)

# d. Bucket performance into categories

In [37]:
def bucket_performance(score):
    if score < 3.0:
        return 'Low'
    elif score < 4.0:
        return 'Medium'
    else:
        return 'High'

In [39]:
merged_df['PerformanceCategory'] = merged_df['AvgScore'].apply(bucket_performance)

# e. Correct mismatched or blank department entries

In [41]:
valid_departments = ['Hr', 'Engineering', 'Sales', 'Marketing', 'Finance']
merged_df['Department'] = merged_df['Department'].apply(lambda d: d if d in valid_departments else 'Other')

In [43]:
print(merged_df[['EmployeeID', 'Name', 'Department', 'Age', 'AvgScore', 'PerformanceCategory', 'EligiblePromotion']])

   EmployeeID  Name   Department   Age  AvgScore PerformanceCategory  \
0         301  John           Hr  34.0      3.65              Medium   
1         302  Jane  Engineering  29.0      4.35                High   
2         303  Mike        Sales  40.0      2.65                 Low   
3         304  Sara        Sales  26.0      4.60                High   

   EligiblePromotion  
0                  1  
1                  1  
2                  0  
3                  1  
